# Span Batch Analysis - Goerli Result Analysis

Get number of channels and each channel names. Channel name must have length 32 in hexadecimal form.

In [49]:
import os
import re

DATA_ROOT_DIR = "./artifacts"
CHAIN_NAME = "goerli"

CHANNEL_CACHE_DIR = f"{DATA_ROOT_DIR}/{CHAIN_NAME}/channel_cache"


def is_hexadecimal(s: str) -> bool:
    hex_pattern = re.compile(r'^[a-fA-F0-9]+$')
    return bool(hex_pattern.match(s))


def read_channel_ids(directory):
    channel_ids = set()
    with os.scandir(directory) as entries:
        for entry in entries:
            assert entry.is_file()
            filename = entry.name
            channel_id, ext = os.path.splitext(filename)
            assert ext == ".json" and len(channel_id) == 32 and is_hexadecimal(channel_id)
            channel_ids.add(channel_id)
    return channel_ids

channel_ids = read_channel_ids(CHANNEL_CACHE_DIR)
channel_id_count = len(channel_ids)
print(f"Number of channels: {channel_id_count}")


Number of channels: 2467


Sanity check of results. Number of jsons in each result directory must be equal to number of channels.

In [50]:
SPAN_BATCH_CACHE_V1_DIR = f"{DATA_ROOT_DIR}/{CHAIN_NAME}/span_batch_cache_v1"
SPAN_BATCH_CACHE_V2_DIR = f"{DATA_ROOT_DIR}/{CHAIN_NAME}/span_batch_cache_v2"
SPAN_BATCH_CACHE_V3_DIR = f"{DATA_ROOT_DIR}/{CHAIN_NAME}/span_batch_cache_v3"
SPAN_BATCH_CACHE_DIRS = [SPAN_BATCH_CACHE_V1_DIR, SPAN_BATCH_CACHE_V2_DIR, SPAN_BATCH_CACHE_V3_DIR]

for directory in SPAN_BATCH_CACHE_DIRS:
    temp_channel_ids = read_channel_ids(directory) 
    assert temp_channel_ids == channel_ids, (len(temp_channel_ids), len(channel_ids))

Sample single channel and observe span batch results.

In [66]:
import random
import json

RESULT_V1_DIR = f"{DATA_ROOT_DIR}/{CHAIN_NAME}/result_v1"
RESULT_V2_DIR = f"{DATA_ROOT_DIR}/{CHAIN_NAME}/result_v2"
RESULT_V3_DIR = f"{DATA_ROOT_DIR}/{CHAIN_NAME}/result_v3"
RESULT_DIRS = [RESULT_V1_DIR, RESULT_V2_DIR, RESULT_V3_DIR]

channel_id = random.choice(list(channel_ids))

results = []
for dir in RESULT_DIRS:
    filename = f"{dir}/{channel_id}.json"
    with open(filename) as f:
        result = json.load(f)
        results.append(result)
        print(result)

{'FrameCount': 1, 'BatchV1sCompressedSize': 12673, 'BatchV1sUncompressedSize': 28587, 'BatchV1sCompressionRatio': 0.4433133942001609, 'SpanBatchCompressedSize': 10910, 'SpanBatchUncompressedSize': 25782, 'SpanBatchCompressionRatio': 0.4231634473663796, 'BatchV1sMetadataSize': 2836, 'BatchV1sTxSize': 25751, 'SpanBatchMetadataSize': 89, 'SpanBatchTxSize': 25693, 'SpanBatchPrefixSize': 48, 'SpanBatchPayloadSize': 25734, 'CompressedReductionPercent': 13.911465319971594, 'UncompressedSizeReductionPercent': 9.812152376954565, 'L2TxCount': 25, 'L1StartNum': 9466703, 'L1EndNum': 9466707, 'L1BlockCount': 5, 'L2StartNum': 12906170, 'L2EndNum': 12906204, 'L2BlockCount': 35}
{'FrameCount': 1, 'BatchV1sCompressedSize': 12673, 'BatchV1sUncompressedSize': 28587, 'BatchV1sCompressionRatio': 0.4433133942001609, 'SpanBatchCompressedSize': 11015, 'SpanBatchUncompressedSize': 25911, 'SpanBatchCompressionRatio': 0.4251090270541469, 'BatchV1sMetadataSize': 2836, 'BatchV1sTxSize': 25751, 'SpanBatchMetadataSi

Plots before compression.

In [67]:
import matplotlib.pyplot as plt
 
index = [f"v{i + 1}" for i in range(3)]
data_batchV0 = {
    f"v{i + 1}": results[i]["BatchV1sUncompressedSize"] for i in range(3)
}
data_spanbatch = {
    f"v{i + 1}": results[i]["SpanBatchUncompressedSize"] for i in range(3)
}
data_reduction_percent = {
    f"v{i + 1}": results[i]["UncompressedSizeReductionPercent"] for i in range(3)
}

fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(111)

ax.bar(x=index, height=data_batchV0.values(), align="center")
for key, value in data_batchV0.items():
    plt.text(x=key, y=value+1, s=f"{value}", fontdict=dict(fontsize=15))

# indicate best tx encoding as green
colors = ["orange" for _ in index]
min_idx = list(data_reduction_percent.values()).index(max(data_reduction_percent.values()))
colors[min_idx] = "green"

ax.bar(x=index, height=data_spanbatch.values(), align="center", color=colors)
for key, value in data_spanbatch.items():
    plt.text(x=key, y=value * 0.9, s=f"{value}", fontdict=dict(fontsize=15))
    percentage = f"{str(100 - data_reduction_percent[key])[:4]} %"
    plt.text(x=key, y=value // 2, s=percentage, fontdict=dict(fontsize=15))

plt.xlabel("tx encode type")
plt.ylabel("byte")
plt.title("uncompressed")
plt.show()

Plots after compression.

In [68]:
import matplotlib.pyplot as plt
 
index = [f"v{i + 1}" for i in range(3)]
data_batchV0 = {
    f"v{i + 1}": results[i]["BatchV1sCompressedSize"] for i in range(3)
}
data_spanbatch = {
    f"v{i + 1}": results[i]["SpanBatchCompressedSize"] for i in range(3)
}
data_reduction_percent = {
    f"v{i + 1}": results[i]["CompressedReductionPercent"] for i in range(3)
}

fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(111)

ax.bar(x=index, height=data_batchV0.values(), align="center")
for key, value in data_batchV0.items():
    plt.text(x=key, y=value+1, s=f"{value}", fontdict=dict(fontsize=15))

# indicate best tx encoding as green
colors = ["orange" for _ in index]
min_idx = list(data_reduction_percent.values()).index(max(data_reduction_percent.values()))
colors[min_idx] = "green"

ax.bar(x=index, height=data_spanbatch.values(), align="center", color=colors)
for key, value in data_spanbatch.items():
    plt.text(x=key, y=value+1, s=f"{value}", fontdict=dict(fontsize=15))
    percentage = f"{str(100 - data_reduction_percent[key])[:4]} %"
    plt.text(x=key, y=value // 2, s=percentage, fontdict=dict(fontsize=15))

plt.xlabel("tx encode type")
plt.ylabel("byte")
plt.title("compressed")
plt.show()